In [2]:
import numpy as np
import os
import json

In [ ]:
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2Model

In [3]:
# Load Data: 

os.chdir('/Users/brandoncheong/Desktop/ML_Dissertation/wiki-pages')

train_data = np.load('train_data.npy', allow_pickle = True)

In [ ]:
train_data  = [json.loads(string.replace('""','\""')) for string in train_data]

In [3]:
print(train_data[0])

{'id': 75397, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'evidence': [[[92206, 104971, 'Nikolaj_Coster-Waldau', 7], [92206, 104971, 'Fox_Broadcasting_Company', 0]]]}


In [4]:
print(len(train_data))

145449


In [7]:
print(train_data[102])

{'id': 113942, 'verifiable': 'NOT VERIFIABLE', 'label': 'NOT ENOUGH INFO', 'claim': 'Bob Dylan has been inducted into four different colleges.', 'evidence': [[[133857, None, None, None]]]}


In [5]:
full_train_data_new = list()

for dictionary in train_data:
 
    for element in dictionary['evidence']:
        
        if len(element) == 0 and dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1:

            continue
            
        elif len(element) == 0 and dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1:
            case = False

            break
            
        elif dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1 and type(element[0]) != str:
            continue
        
            
        
        elif dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1 and type(element[0]) != str:
            case = False

            break
        
        else:
            
            case = True

            idx = dictionary['evidence'].index(element) 
            
    
    if case == True and idx == len(dictionary['evidence']) - 1:
        
            
        text = dictionary['evidence'][idx]        
        
        alt_dictionary = copy.deepcopy(dictionary)
        alt_dictionary['evidence'] = text[0]
        full_train_data_new.append(alt_dictionary)
    
    
    elif case == True and idx < len(dictionary['evidence']) - 1:  
        
        texts = dictionary['evidence'][idx:] # adapt to specific structure of the data: this is a list of lists
        initial_text = texts[0]
        for text in texts[1:]:
            initial_text += text
        
        alt_dictionary = copy.deepcopy(dictionary)

        alt_dictionary['evidence'] = initial_text
        full_train_data_new.append(alt_dictionary)
    
    else: # idx does not exist
        continue        

In [10]:
print(full_train_data_new[0])

IndexError: list index out of range

In [ ]:
# NEI time:
from transformers import pipeline
import copy
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")
unmasker = pipeline('fill-mask', model='bert-base-cased')


# obtain full dataset consisting of original claims + Bert-generated evidence_statements:

bert_nei_data = list()
for dictionary in full_train_data_new:
    if dictionary['label'] == 'NOT ENOUGH INFO':
           entry = copy.deepcopy(dictionary)
           NER_on_claim = NER(entry['claim'])
    # set all claims to 'empty' if not appropriate - will skip this in training data
           if len(list(NER_on_claim.ents)) == 0:
                entry['claim'] = 'Empty'
                continue
           else: 
                masked_entity = list(NER_on_claim.ents)[-1].text
                masked_entity_start_idx = entry['claim'].index(masked_entity)
                masked_entity_end_idx = masked_entity_start_idx + len(masked_entity) + 1
                text = entry['claim'][:masked_entity_start_idx] + '[MASK]' + entry['claim'][masked_entity_end_idx:]
                if text[-1] == '.':
                       pass
                else:
                       text += '.'
                print(text)
                prediction = unmasker(text)[0]['token_str']
                print(prediction)
                entry['evidence'] = entry['claim'][:masked_entity_start_idx] + prediction + entry['claim'][masked_entity_end_idx:]
                bert_nei_data.append(entry) 
                print(f'Entry {baseline_20_train_data.index(entry)} done!')


In [ ]:
# Now, time to work on GPT-2: 

masked_train_data = copy.deepcopy(full_train_data_new)

for entry in masked_train_data:
    NER_on_claim = NER(entry['claim'])
    if len(list(NER_on_claim.ents)) == 0:
        entry['claim'] = 'Empty'
        continue
    else: 
        masked_entity = list(NER_on_claim.ents)[-1].text
    masked_entity_idx = entry['claim'].index(masked_entity)
    entry['claim'] = entry['claim'][:masked_entity_idx]
    print(f'Entry {masked_train_data.index(entry)} done!')

In [ ]:
# train GPT2 on Fever by generating evidence_statements (prompts):

# Note: max_length, num_return_seq both hyperparameters - can fine-tune!

generator = pipeline('text-generation', model='gpt2')


full_GPT2_train_data = copy.deepcopy(masked_train_data)
set_seed(42)


for entry in full_GPT2_train_data:
    prompt = entry['claim']
    ev_statement = generator(prompt, max_length=50, num_return_sequences=1)[0]
    entry['evidence'] = ev_statement
    print(f'Done with {full_GPT2_train_data.index(entry)}th entry')

In [ ]:
# final step
for i in range(len(full_GPT2_train_data)):
    
    full_GPT2_train_data[i]['claim'] = full_train_data_new[i]['claim']
    full_GPT2_train_data[i]['evidence'] = full_GPT2_train_data[i]['evidence']['generated_text']